## NIPS 2018 Hierarchical Graph Representation Learning with Differentiable Pooling
- 图卷积的分层表示学习方法，主要是学得图的表示之后进行图的分类

![](img/HGRL.png)

### 图卷积

$$\begin{eqnarray*}H^{k}&=M(A,H^{k-1},\theta^{k}) \in R^{n \times d}\tag{1} \\
M(A,H^{k-1}，W^{k}) &= ReLU(\tilde{D} ^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}} H^{k-1}W^{k-1}) \tag{2}
\end{eqnarray*}$$

其中 $\tilde{A}=A+I ,\tilde{D}=\sum_{j}\tilde{A}_{ij},W^{k} \in R^{d \times d}, H^{k} \in R^{n \times d}$ $W$为待学习的参数,$A$为图的邻接矩阵。卷积的过程可以看做是消息在每个节点按相邻点进行扩散的过程，因为邻接矩阵包含了每个点的邻居的信息。

这里$A+I$是考虑到节点自身的影响，另外$\tilde{D}$主要是为了进行归一化处理，使$A$的每一行的为1。

可以将上面的过程简化为$Z=GNN(A,X)$

### 通过学习任务设计可以微分的池化

$$\begin{eqnarray*} X_{l+1} &= S_{l}^{T}Z_{l}  \in R^{n_{l+1} \times d} \tag{3} \\
A_{l+1} &= S_{l}^{T}A_{l}S_{l} \in R^{n_{l+1} \times n_{l+1}} \tag{4}\\
\end{eqnarray*}$$

其中$S_{l} \in R^{n_{l} \times n_{l+1}}$, 可以理解为当前层跟下一次cluster之间的相关性，是需要学习的参数

$$\begin{eqnarray*} Z_{l} &= GNN_{l,embed}(A_{l},X_{l}) \tag{5} \\
S_{l} &= softmax(GNN_{l,pool}(A_{l},X_{l})) \tag{6}
\end{eqnarray*}$$

- 这里softmax的作用主要是将$l$层每个节点对$l+1$的贡献进行归一化。
- 这样随着层数的依次增加，cluster会逐渐减小，直到成为一个节点，那么用最后一个节点表示的向量传入另外一个神经网络中进行分类或者其他任务

### 辅助目标函数
- 一个是分类的目标函数，另外两个: 一个是进行聚合的时候应该尽量让相近的节点聚在一起，一个是进行softmax的时候使用交叉熵损失，因为我们希望的是$l$和$l+1$层是一种单射的关系。